In [2]:
import re
import sys
import shutil as sh
import os

In [3]:
path = "/home/yingjie/Desktop/Machine_Learning/Assignment4/data/"
path2 = "/home/yingjie/Desktop/Machine_Learning/Assignment4/data_clean/"
#path = "/Users/Pan/Google Drive/Data Science/Machine Learning/ML_Assignment4/data/"
#path2 = "/Users/Pan/Google Drive/Data Science/Machine Learning/ML_Assignment4/data_clean/"

In [4]:
def data_preprocess(path,thefile,num_of_reviews=None):    
    # Replace all numbers with "NUM" special character
    num_dec = re.compile("[\d]+\.[\d]+")
    num_comma = re.compile("[\d]+,[\d]+")
    num_reg = re.compile("[\d]+")

    # Detect non-English characters
    non_eng = re.compile("[^a-zA-Z\n ]")
    
    #open file
    file=open(os.path.join(path,thefile), "r", encoding="iso-8859-15")
    text = file.read().lower()
    file.close()
    
    # Replace various forms of numbers with NUM character
    text = num_dec.sub("NUM", text)
    text = num_comma.sub("NUM", text)
    text = num_reg.sub("NUM", text)

    # Replace all non-English characters with a space
    text = non_eng.sub(" ", text)

    # Replace double spaces with a single space
    text = text.replace("  ", " ")
    
    # Split the file into reviews based on \n
    text = text.split("\n")
    
    if num_of_reviews is None:
        num_of_reviews=len(text)
        
    small_train=''
    for i in range(num_of_reviews):
        small_train+=text[i]+" END "
    small_train=[small_train]
    
    return(small_train)

In [5]:
#full
train=data_preprocess(path,"down_sampled_reviews/train_tiny.txt")
valid=data_preprocess(path,"down_sampled_reviews/valid_tiny.txt")

In [6]:
#partial
train=data_preprocess(path,"down_sampled_reviews/train_tiny.txt",num_of_reviews=50)
valid=data_preprocess(path,"down_sampled_reviews/valid_tiny.txt",num_of_reviews=10)

## Convert Train to IDs

In [7]:
import numpy as np
from tensorflow.contrib import learn

/home/yingjie/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
def train_textlst_2_vocab(text,seq_size = 2):   
    max_document_length=len(text[0].split(" "))
    
    ## Create the vocabularyprocessor object, setting the max lengh of the documents.
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)   

    ## Transform the documents using the vocabulary.
    train_transformed = np.array(list(vocab_processor.fit_transform(text)))
    
    vocab_dict = vocab_processor.vocabulary_._mapping
    vocab_dict['END'] = -1

    ## Extract word:id mapping from the object.
    vocab_dict = vocab_processor.vocabulary_

    train_transformed=np.array(list(vocab_processor.fit_transform(text)))
    train_transformed=[np.trim_zeros(train_transformed[0], 'b')] #trim zero at the end
    
    trainX, trainY = [], []
    max_document_length=len(train_transformed[0])
    for i in range(max_document_length): 
        #[0,1,2,3,4,5,-1,1,2,3,4,5]
        if -1 not in train_transformed[0][i:i+seq_size+1]:
            trainX.append(np.expand_dims(train_transformed[0][i:i+seq_size], axis=1).tolist())
            trainY.append(train_transformed[0][i+1:i+seq_size+1])
    
    size_vocab = len(vocab_dict)
    
    return trainX,trainY,size_vocab,vocab_dict

In [9]:
seq_size=2
trainX,trainY,size_vocab,train_vocab=train_textlst_2_vocab(train,seq_size = seq_size)
print(size_vocab)
print("X:\n",'\n'.join([str(item) for item in trainX[0:5]]))
print("Y:\n",'\n'.join([str(item) for item in trainY[0:5]]))

2283
X:
 [[1], [2]]
[[2], [3]]
[[3], [4]]
[[4], [5]]
[[5], [6]]
Y:
 [2 3]
[3 4]
[4 5]
[5 6]
[6 7]


## Convert Valid to IDs

In [10]:
max_document_length=len(train[0].split(" "))

In [11]:
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length,vocabulary=train_vocab)

## Transform the documents using the vocabulary.
train_transformed =np.array(list(vocab_processor.fit_transform(valid)))
train_transformed=[np.trim_zeros(train_transformed[0], 'b')] #trim zero at the end

## Extract word:id mapping from the object.
valid_vocab = vocab_processor.vocabulary_

validX, validY = [], []

for i in range(max_document_length): 
    #[0,1,2,3,4,5,-1,1,2,3,4,5]
    if -1 not in train_transformed[0][i:i+seq_size+1]:
        validX.append(np.expand_dims(train_transformed[0][i:i+seq_size], axis=1).tolist())
        validY.append(train_transformed[0][i+1:i+seq_size+1])

In [12]:
print(valid_vocab._mapping.items())
print(valid_vocab._mapping.items()==train_vocab._mapping.items()) #if match then good
print(size_vocab)
print("X:\n",'\n'.join([str(item) for item in validX[0:5]]))
print("Y:\n",'\n'.join([str(item) for item in validY[0:5]]))

dict_items([('<UNK>', 0), ('i', 1), ('don', 2), ('t', 3), ('know', 4), ('why', 5), ('but', 6), ('flexaril', 7), ('just', 8), ('isn', 9), ('doing', 10), ('the', 11), ('trick', 12), ('for', 13), ('me', 14), ('anymore', 15), ('m', 16), ('shocked', 17), ('by', 18), ('how', 19), ('well', 20), ('this', 21), ('product', 22), ('works', 23), ('muscle', 24), ('spasms', 25), ('suppose', 26), ('valerian', 27), ('and', 28), ('passiflora', 29), ('which', 30), ('are', 31), ('also', 32), ('mental', 33), ('relaxants', 34), ('along', 35), ('with', 36), ('magnesium', 37), ('cramping', 38), ('help', 39), ('give', 40), ('you', 41), ('that', 42), ('over', 43), ('all', 44), ('feeling', 45), ('of', 46), ('ahhh', 47), ('not', 48), ('everything', 49), ('everyone', 50), ('it', 51), ('s', 52), ('a', 53), ('relatively', 54), ('expensive', 55), ('purchase', 56), ('if', 57), ('doesn', 58), ('re', 59), ('need', 60), ('one', 61), ('their', 62), ('other', 63), ('vitamins', 64), ('order', 65), ('those', 66), ('they', 67

## TF SETUP

In [13]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib import rnn
from sklearn.preprocessing import MinMaxScaler
import timeit

In [14]:
hidden_dim=100
max_iter=10
cellType='rnn'

In [15]:
graph = tf.Graph()
with graph.as_default():
    # input place holders
    # input Shape: [# training examples, sequence length, # features]
    x = tf.placeholder(tf.float32,[None,seq_size,1])
    print("input shape:[# training examples, sequence length, # features]", x.shape)

    # label Shape: [# training examples, sequence length]
    y = tf.placeholder(tf.int32,[None,seq_size])
    print("ground truth shape:[# training examples, sequence length]", y.shape)

    # RNN Network
    cell = rnn.BasicRNNCell(hidden_dim)

    # RNN output Shape: [# training examples * sequence length, # hidden] 
    outputs, _ = tf.nn.dynamic_rnn(cell,x,dtype=tf.float32)
    num_examples = tf.shape(x)[0] 

    outputs=tf.reshape(outputs,[num_examples*seq_size, hidden_dim])
    print("output shape: [# training examples, sequence length, # hidden] ", outputs.shape)
    
    # weights for output dense layer (i.e., after RNN)
    # W shape: [# hidden, 1]
    W_out = tf.Variable(tf.random_normal([hidden_dim, size_vocab]),name="w_out") 
    print("weights shape: [# hidden, 1]", W_out.shape)
    
    # b shape: [1]
    b_out = tf.Variable(tf.random_normal([size_vocab]),name="b_out")
    print("bias shape [1]",b_out.shape)
    
    y_pred = tf.matmul(outputs,W_out)+b_out
    y_pred=tf.reshape(y_pred,[num_examples, seq_size, size_vocab])
    print("final logits shape:[# training examples, vocab_size] ", y_pred.shape)

    # Cost & Training Step
    cost = tf.contrib.seq2seq.sequence_loss(y_pred,
                                            y,
                                            tf.ones([num_examples,seq_size],tf.float32),
                                            average_across_timesteps=False,
                                            average_across_batch=False)
    cost = tf.reduce_mean(cost) 
    train_op = tf.train.AdamOptimizer().minimize(cost)
    saver=tf.train.Saver()


input shape:[# training examples, sequence length, # features] (?, 2, 1)
ground truth shape:[# training examples, sequence length] (?, 2)
output shape: [# training examples, sequence length, # hidden]  (?, 100)
weights shape: [# hidden, 1] (100, 2283)
bias shape [1] (2283,)
final logits shape:[# training examples, vocab_size]  (?, 2, 2283)


In [17]:
# Run Session
with tf.Session(graph=graph) as sess:
    # initialize variables
    sess.run(tf.global_variables_initializer())
    # Run for 1000 iterations (1000 is arbitrary, need a validation set to tune!)
    start=timeit.default_timer()
    print('Training %s ...'%cellType)
    train_c = 0
    for i in range(max_iter): # If we train more, would we overfit? Try 10000
        #
        print("iter %3d" %(i)) 
        
        saver_train_error = 0
        for NUM in range(1,batch_num):
            batch_trainX=trainX[(NUM-1)*batch_size:NUM*batch_size]
            batch_trainY=trainY[(NUM-1)*batch_size:NUM*batch_size]
        
            _,train_err= sess.run([train_op,cost],feed_dict={x:trainX,y:trainY})
            train_c += train_err
            mean_train_error = train_c/NUM
            
            if  NUM % 100 == 0: 
                print("  ##batch NUM %6d :" %(NUM))
                print('    step, train err= %6d: %8.5f' % (i+1,train_err))

        if i==0:
            print('  step, train err= %6d: %8.5f' % (0,mean_train_error)) 
        elif  (i+1) % 2 == 0: 
            print('  step, train err= %6d: %8.5f' % (i+1,mean_train_error))
            
        print('  interation, testing error= %6d: %8.5f, \
        sess.run(cost, feed_dict={x: validX[0:800],
                                      y: validY[0:800]}))
    end=timeit.default_timer()        
    print("Training time : %10.5f"%(end-start))

Training rnn ...
  step, train err=      0: 36.05583
  step, train err=      2: 35.51660
  step, train err=      4: 34.60969
  step, train err=      6: 33.40064
  step, train err=      8: 32.26937
  step, train err=     10: 31.20195
Testing Accuracy: 27.695852
Training time :   14.57631


In [99]:
len(validX)

8611

In [100]:
len(validY)

8611

In [101]:
len(trainX)

8338

In [102]:
len(trainY)

8338